# Visualizing Clusters of Clickbait Headlines Using Spark and Word2vec

by Max Woolf (@minimaxir)

*This notebook is licensed under the MIT License. If you use the code or data visualization designs contained within this notebook, it would be greatly appreciated if proper attribution is given back to this notebook and/or myself. Thanks! :)*

In [1]:
spark_path = "/Users/maxwoolf/Desktop/spark-2.0.0-bin-hadoop2.7"

# Easiest way to get Spark to work with Jupyter: https://github.com/minrk/findspark
import findspark
findspark.init(spark_path)

import pyspark
from pyspark.sql import SparkSession
sc = pyspark.SparkContext(appName='fb_headlines')
spark = SparkSession(sc)

In [2]:
config = sc._conf.getAll()
print 'Web URL: ' + filter(lambda x: 'spark.driver.host' in x[0], config)[0][1] + ':4040'

Web URL: 10.0.1.11:4040


Add a function to preprocess each DF.

In [3]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql import Column
from pyspark.sql.types import *



def process_df(df):
    return (df
        .withColumn('text',regexp_replace('link_name', '\'', ''))
      .filter("text != ''")
      .filter("text != 'Timeline Photos'")
           )

def read_tsv(path):
    
    # predefine data frame schema: faster
    
    schema = StructType([
                StructField("page_id", StringType(), True),
                StructField("status_id", StringType(), True),
                StructField("link_name", StringType(), True),
                StructField("status_published", TimestampType(), True),
                StructField("num_reactions", IntegerType(), True)])
    
    return spark.read.csv(path ,schema = schema, header=True, inferSchema=True, sep="\t")

In [4]:
df_cnn = read_tsv("fb_headlines/CNN_fb.tsv")
df_nytimes = read_tsv("fb_headlines/NYTimes_fb.tsv")
df_buzzfeed = read_tsv("fb_headlines/BuzzFeed_fb.tsv")
df_upworthy = read_tsv("fb_headlines/Upworthy_fb.tsv")

df = df_cnn.union(df_nytimes).union(df_buzzfeed).union(df_upworthy)
df = process_df(df).cache()

df.select('link_name', 'status_published', 'num_reactions').show()
print df.count()
df.printSchema()

+--------------------+--------------------+-------------+
|           link_name|    status_published|num_reactions|
+--------------------+--------------------+-------------+
|Joseph Schooling ...|2016-08-12 21:14:...|         3508|
|Bill Clinton: Ema...|2016-08-12 21:01:...|         1592|
|Hacker releases c...|2016-08-12 21:01:...|          446|
|Lionel Messi anno...|2016-08-12 20:39:...|         1324|
|Fighting the male...|2016-08-12 20:31:...|          129|
|The face of the O...|2016-08-12 20:11:...|         1597|
|Trump: If Clinton...|2016-08-12 19:51:...|         4017|
|Texas baby found ...|2016-08-12 19:28:...|         4860|
|Malawi is moving ...|2016-08-12 19:01:...|          310|
|Thomas Gibson fir...|2016-08-12 18:44:...|         2026|
|Watch Donald Trum...|2016-08-12 18:01:...|         2600|
|Watch a timelapse...|2016-08-12 17:22:...|         1299|
|Poll: Clinton lea...|2016-08-12 16:46:...|         9445|
|Brendan Dassey's ...|2016-08-12 16:19:...|         9240|
|Live from Yan

In [5]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

tokenizer = RegexTokenizer(pattern="[^\w]",inputCol="text", outputCol="words")
df = tokenizer.transform(df)
df.select('text','words').show()

+--------------------+--------------------+
|                text|               words|
+--------------------+--------------------+
|Joseph Schooling ...|[joseph, schoolin...|
|Bill Clinton: Ema...|[bill, clinton, e...|
|Hacker releases c...|[hacker, releases...|
|Lionel Messi anno...|[lionel, messi, a...|
|Fighting the male...|[fighting, the, m...|
|The face of the O...|[the, face, of, t...|
|Trump: If Clinton...|[trump, if, clint...|
|Texas baby found ...|[texas, baby, fou...|
|Malawi is moving ...|[malawi, is, movi...|
|Thomas Gibson fir...|[thomas, gibson, ...|
|Watch Donald Trum...|[watch, donald, t...|
|Watch a timelapse...|[watch, a, timela...|
|Poll: Clinton lea...|[poll, clinton, l...|
|Brendan Dasseys c...|[brendan, dasseys...|
|Live from Yankee ...|[live, from, yank...|
|Live: The story b...|[live, the, story...|
|US womens soccer ...|[us, womens, socc...|
|There are no winn...|[there, are, no, ...|
|Senate Republican...|[senate, republic...|
|Live: Flooding in...|[live, flo

In [6]:
from pyspark.ml.feature import Word2Vec

word2Vec = Word2Vec(vectorSize=50, seed=42, inputCol="words", outputCol="vectors")
model = word2Vec.fit(df)
df = model.transform(df)

df.select('words', 'vectors').show()

+--------------------+--------------------+
|               words|             vectors|
+--------------------+--------------------+
|[joseph, schoolin...|[-0.0295360057022...|
|[bill, clinton, e...|[0.00839712098240...|
|[hacker, releases...|[-0.0453028933377...|
|[lionel, messi, a...|[-0.0399187423288...|
|[fighting, the, m...|[-0.0419914640951...|
|[the, face, of, t...|[-0.0460251685231...|
|[trump, if, clint...|[0.00891906013047...|
|[texas, baby, fou...|[-0.0943986974656...|
|[malawi, is, movi...|[0.04506426247826...|
|[thomas, gibson, ...|[-0.1057210512226...|
|[watch, donald, t...|[-0.0874061348537...|
|[watch, a, timela...|[0.06606105796527...|
|[poll, clinton, l...|[0.06775298264498...|
|[brendan, dasseys...|[0.04835192763776...|
|[live, from, yank...|[-0.0411922155568...|
|[live, the, story...|[-0.0622380969247...|
|[us, womens, socc...|[-0.0382462517757...|
|[there, are, no, ...|[0.07911438743273...|
|[senate, republic...|[-0.0546692525036...|
|[live, flooding, ...|[0.0260696

Not required for analysis, but fun to check!

In [7]:
model.findSynonyms("olympics", 20).show()
model.findSynonyms("election", 20).show()

+-----------+------------------+
|       word|        similarity|
+-----------+------------------+
|       deer|0.7518758500661575|
| apocalypse|0.7027730099556968|
|   happiest|0.6720302546187348|
|       vmas|0.6665068107149325|
|     upshot|0.6659036225560672|
|       grid|0.6645280176415026|
| reclaiming|0.6620794275524378|
|    opening|0.6545831443299779|
|opinionator|0.6539698447594439|
|     oscars|0.6491699200786258|
|  countdown|0.6488588551173281|
|       wild|0.6446697158905897|
|    extreme| 0.633502635230153|
|    largest|  0.63119806576113|
|     carpet|0.6290782842323763|
|   ceremony|0.6275797991993366|
|     simone|0.6215671608391896|
|    longest|0.6200576886880117|
|  brightest|0.6186380473151332|
|   smallest|0.6142787876081013|
+-----------+------------------+

+------------+------------------+
|        word|        similarity|
+------------+------------------+
|presidential|0.7822506427244476|
|  democratic|0.7412928207831224|
|  republican|0.7373115162062022|
|  

In [8]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

stringIndexer = StringIndexer(inputCol="page_id", outputCol="indexed")
model = stringIndexer.fit(df)
df = model.transform(df)
encoder = OneHotEncoder(inputCol="indexed", outputCol="page_ohe")
df = encoder.transform(df)

df.select('page_id', 'page_ohe').show()

+-------+-------------+
|page_id|     page_ohe|
+-------+-------------+
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
|    CNN|(3,[1],[1.0])|
+-------+-------------+
only showing top 20 rows



In [9]:
from pyspark.ml.feature import VectorAssembler

model = VectorAssembler(inputCols=['page_ohe', 'vectors'], outputCol="merged_vectors")
df = model.transform(df)

df.select('merged_vectors').show()

+--------------------+
|      merged_vectors|
+--------------------+
|[0.0,1.0,0.0,-0.0...|
|[0.0,1.0,0.0,0.00...|
|[0.0,1.0,0.0,-0.0...|
|[0.0,1.0,0.0,-0.0...|
|[0.0,1.0,0.0,-0.0...|
|[0.0,1.0,0.0,-0.0...|
|[0.0,1.0,0.0,0.00...|
|[0.0,1.0,0.0,-0.0...|
|[0.0,1.0,0.0,0.04...|
|[0.0,1.0,0.0,-0.1...|
|[0.0,1.0,0.0,-0.0...|
|[0.0,1.0,0.0,0.06...|
|[0.0,1.0,0.0,0.06...|
|[0.0,1.0,0.0,0.04...|
|[0.0,1.0,0.0,-0.0...|
|[0.0,1.0,0.0,-0.0...|
|[0.0,1.0,0.0,-0.0...|
|[0.0,1.0,0.0,0.07...|
|[0.0,1.0,0.0,-0.0...|
|[0.0,1.0,0.0,0.02...|
+--------------------+
only showing top 20 rows



Collect subset of post since June to make visualization more manageable.

In [10]:
df2 = df.filter(df.status_published > '2016-06-01 00:00:00').cache()

print df2.count()
df2.printSchema()

9500
root
 |-- page_id: string (nullable = true)
 |-- status_id: string (nullable = true)
 |-- link_name: string (nullable = true)
 |-- status_published: timestamp (nullable = true)
 |-- num_reactions: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- vectors: vector (nullable = true)
 |-- indexed: double (nullable = true)
 |-- page_ohe: vector (nullable = true)
 |-- merged_vectors: vector (nullable = true)



To CSV, for passing into R.

In [11]:
import pandas as pd

(df2.select('page_id', 'status_id', 'link_name', 'status_published', 'num_reactions', 'merged_vectors')
    .toPandas()
    .to_csv('fb_headlines_53D.csv', index=False, encoding="utf-8")
 )

# The MIT License (MIT)

Copyright (c) 2016 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.